In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller

import warnings
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

from data_import import *
from modeling_prep import *

##### Data Import

In [2]:
oregon_data_dict = oregon_import(float_32=True)

file import: 100%|██████████| 3/3 [00:01<00:00,  2.74it/s]


##### Examining data suitability for VARMAX

Verifying stationarity of data

In [8]:
#looking at one county in particular

wa_df = oregon_data_dict['train_timeseries'].copy()
wa_df = wa_df[wa_df['fips']==41067]
wa_df.drop(columns=['fips'],inplace=True)
wa_df = wa_df.iloc[4:,:]
wa_df = wa_df.iloc[:-4,:]
wa_df['date'] = wa_df['date'].map(pd.Timestamp.timestamp)
wa_df.reset_index(inplace=True,drop=True)
wa_county_score_list = wa_df['score'].dropna().copy()


result = adfuller(wa_county_score_list)
print('ADF Statistic:', result[0])
print('p-value:', result[1])

ADF Statistic: -5.508469601022172
p-value: 1.9969356496943797e-06


In [7]:
oregon_df = oregon_data_dict['train_timeseries'].copy()
oregon_score_list = oregon_df['score'].dropna().copy()

result = adfuller(oregon_score_list)
print('ADF Statistic:', result[0])
print('p-value:', result[1])

ADF Statistic: -15.628872219809024
p-value: 1.695245873408861e-28


##### VARMAX

In [6]:
train_copy = oregon_data_dict['train_timeseries'].copy()
train_copy.dropna(subset=['score'], how='all', inplace=True)

In [9]:
train_data = pd.read_csv('.\processed_data\oregon_train_timeseries.csv',header=0, index_col=1)

split_data_dict = train_test_split_default(train_data)

In [7]:
import statsmodels.api as sm

In [15]:
len(train_data.iloc[:,train_data.columns != 'split'].columns)

19

In [22]:
train_data['score'].values.reshape(-1,1)

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [ ]:
exog = train_data.iloc[:,train_data.columns != 'split'].copy()
mod = sm.tsa.VARMAX(endog = train_data['score'].values.reshape(-1,1), order=(2,0), trend='n', exog=exog)
res = mod.fit(maxiter=1000, disp=False)
print(res.summary())

c:\Users\pears\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
c:\Users\pears\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\pears\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


ValueError: Only gave one variable to VAR

In [31]:
train_df = train_data.copy()
train_df.index = pd.DatetimeIndex(train_df.index).to_period('M')

In [ ]:
mod = sm.tsa.arima.ARIMA(endog = train_df['score'].values.reshape(-1,1), order=(1, 0, 0), exog = train_df.iloc[:,train_df.columns != 'score'].copy())
test = mod.fit()

c:\Users\pears\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\pears\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\pears\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [ ]:
print(test.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                31932
Model:                 ARIMA(1, 0, 0)   Log Likelihood              338286.962
Date:                Mon, 17 Mar 2025   AIC                        -676529.924
Time:                        14:50:50   BIC                        -676345.754
Sample:                             0   HQIC                       -676471.006
                              - 31932                                         
Covariance Type:                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -8.715e-15   1.46e-25  -5.95e+10      0.000   -8.72e-15   -8.72e-15
PRECTOT      9.107e-18   5.58e-24   1.63e+06      0.000    9.11e-18    9.11e-18
PS           4.337e-18   1.38e-23   3.13e+05    

##### Divider

In [ ]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		y_hat = model_fit.forecast()[0]
		predictions.append(y_hat)
		history.append(test[t])
	# calculate out of sample error
	rmse = np.sqrt(mean_squared_error(test, predictions))
	return rmse

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	# dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				# try:
				rmse = evaluate_arima_model(dataset, order)
				if rmse < best_score:
					best_score, best_cfg = rmse, order
				print('ARIMA%s RMSE=%.3f' % (order,rmse))
				# except:
					# continue
	print('Best ARIMA %s RMSE = %.3f' % (best_cfg, best_score))


# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(train_data.values, p_values, d_values, q_values)

##### Modeling

In [ ]:
train_copy = oregon_data_dict['train_timeseries'].copy()
train_copy.dropna(subset=['score'], how='all', inplace=True)

In [4]:
model = ARIMA(train_copy, order=(1, 0, 1))
model_fit = model.fit()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:


def parser(x):
	return pd.Datetime.strptime('190'+x, '%Y-%m')

series = pd.read_csv('./processed_data/oregon_train_timeseries.csv', header=0, parse_dates=[0], index_col=0, date_parser=parser)
autocorrelation_plot(series)
plt.show()

C:\Users\pears\AppData\Local\Temp\ipykernel_21744\304317978.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  series = pd.read_csv('./processed_data/oregon_train_timeseries.csv', header=0, parse_dates=[0], index_col=0)


##### Grid Search of ARIMA order values

In [3]:
train_data = pd.read_csv('.\processed_data\oregon_train_timeseries.csv',header=0, index_col=1)

split_data_dict = train_test_split_default(train_data)

In [59]:
grouped_train_data = pd.read_csv('.\processed_data\oregon_train_timeseries.csv',header=0, index_col=1)

split_county_data = county_grouped_shufflesplit(grouped_train_data)

In [ ]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		y_hat = model_fit.forecast()[0]
		predictions.append(y_hat)
		history.append(test[t])
	# calculate out of sample error
	rmse = np.sqrt(mean_squared_error(test, predictions))
	return rmse

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	# dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				# try:
				rmse = evaluate_arima_model(dataset, order)
				if rmse < best_score:
					best_score, best_cfg = rmse, order
				print('ARIMA%s RMSE=%.3f' % (order,rmse))
				# except:
					# continue
	print('Best ARIMA %s RMSE = %.3f' % (best_cfg, best_score))


# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(train_data.values, p_values, d_values, q_values)

AttributeError: 'list' object has no attribute 'score'